In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from itertools import combinations

In [3]:
number_tickets = 50
seating_rows = 20
seating_columns = 20
number_seats = seating_rows * seating_columns
seat_depth = 0.5
seat_depth_gap = 0.2
seat_width = 0.5
seat_width_gap = 0.2

In [4]:
x_seats = np.array( [ (i % seating_columns) * (seat_width + seat_width_gap) + (seat_width / 2) for i in range(number_seats) ] )

In [5]:
y_seats = np.array( [ (i // seating_columns) * (seat_depth + seat_depth_gap) + (seat_depth / 2) for i in range(number_seats) ] )

In [6]:
ticket_size = np.random.randint(low =1, high = 10, size = number_tickets)

In [7]:
ticket_size

array([2, 8, 2, 1, 3, 6, 1, 4, 3, 3, 6, 6, 1, 1, 4, 1, 4, 2, 5, 3, 2, 7,
       8, 7, 3, 5, 9, 4, 3, 5, 1, 1, 9, 6, 8, 3, 9, 8, 9, 6, 5, 9, 3, 9,
       4, 7, 5, 4, 7, 7])

In [11]:
def main():
    # Data
    
        
    
    # Solver
    # Create the mip solver with the CBC backend.
    solver = pywraplp.Solver.CreateSolver('assignment_mip', 'CBC')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(number_seats):
        for j in range(number_tickets):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each ticekt is assigned to only one seat max
    for i in range(number_seats):
        solver.Add(solver.Sum([x[i, j] for j in range(number_tickets)]) <= 1)
    
    for j in range(number_tickets):
        solver.Add(solver.Sum([x[i, j] for i in range(number_seats)]) == 1)
        
    # seating distance maintained
    for m, n in combinations(range(number_tickets), 2):
        solver.Add(
                    (
                       ((solver.Sum([ x[i,m] * x_seats[i] for i in range(number_seats)]) 
                       - solver.Sum([ x[i,n] * x_seats[i] for i in range(number_seats)]) ) *  # change to square or do fancy stuff
                        (solver.Sum([ x[i,m] * x_seats[i] for i in range(number_seats)]) 
                       - solver.Sum([ x[i,n] * x_seats[i] for i in range(number_seats)]) ))
                        +
                        ((solver.Sum([ x[i,m] * y_seats[i] for i in range(number_seats)]) 
                       - solver.Sum([ x[i,n] * y_seats[i] for i in range(number_seats)]) ) * 
                        (solver.Sum([ x[i,m] * y_seats[i] for i in range(number_seats)]) 
                       - solver.Sum([ x[i,n] * y_seats[i] for i in range(number_seats)]) ))
                        
                    ) >= 4
                )

    

    # Objective
    objective_terms = []
    for i in range(number_seats):
        for j in range(number_tickets):
            objective_terms.append(ticket_size[j] * x[i, j])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(number_seats):
            for j in range(number_tickets):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('Worker %d assigned to task %d.  Cost = %d' %
                          #(i, j, costs[i][j]))
                        print(i,j)
    else:
        return status


if __name__ == '__main__':
    soln = main()

TypeError: 

array([ 0.25,  0.95,  1.65,  2.35,  3.05,  3.75,  4.45,  5.15,  5.85,
        6.55,  7.25,  7.95,  8.65,  9.35, 10.05, 10.75, 11.45, 12.15,
       12.85, 13.55,  0.25,  0.95,  1.65,  2.35,  3.05,  3.75,  4.45,
        5.15,  5.85,  6.55,  7.25,  7.95,  8.65,  9.35, 10.05, 10.75,
       11.45, 12.15, 12.85, 13.55,  0.25,  0.95,  1.65,  2.35,  3.05,
        3.75,  4.45,  5.15,  5.85,  6.55,  7.25,  7.95,  8.65,  9.35,
       10.05, 10.75, 11.45, 12.15, 12.85, 13.55,  0.25,  0.95,  1.65,
        2.35,  3.05,  3.75,  4.45,  5.15,  5.85,  6.55,  7.25,  7.95,
        8.65,  9.35, 10.05, 10.75, 11.45, 12.15, 12.85, 13.55,  0.25,
        0.95,  1.65,  2.35,  3.05,  3.75,  4.45,  5.15,  5.85,  6.55,
        7.25,  7.95,  8.65,  9.35, 10.05, 10.75, 11.45, 12.15, 12.85,
       13.55,  0.25,  0.95,  1.65,  2.35,  3.05,  3.75,  4.45,  5.15,
        5.85,  6.55,  7.25,  7.95,  8.65,  9.35, 10.05, 10.75, 11.45,
       12.15, 12.85, 13.55,  0.25,  0.95,  1.65,  2.35,  3.05,  3.75,
        4.45,  5.15,